In [36]:
import matplotlib.pyplot as plt
from mlxtend.data import loadlocal_mnist
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

In [37]:
# images are 28x28
# each row in the np imgs is 784-long
imgs, labels = loadlocal_mnist(
        images_path='/lab/vislab/DATA/just/mnist/train-images-idx3-ubyte', 
        labels_path='/lab/vislab/DATA/just/mnist/train-labels-idx1-ubyte')


# Batch size for training (change depending on how much memory you have)
batch_size = 60

# Type of Loss, not sure what this does yet
criterion = nn.CrossEntropyLoss()

# Connector to GPU
device = torch.device("cuda:0")

# Sqaure size of each training image
img_size = 28

# Number of classes in the dataset
num_classes = 10

# Number of epochs to train for
num_epochs = 25

In [48]:
class Model_(nn.Module):
    """Basic ResNet18 model for this set. Any changes or suggestions are welcome"""
    
    def __init__(self):
        super(Model_, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(img_size, num_classes).to('cuda:0')  # 6*6 from image dimension
        self.fc2 = nn.Linear(num_classes, num_classes).to('cuda:0')
#         self.fc=nn.Linear(img_size, num_classes).to('cuda:0')
        
    def forward(self, x):
        """Forward propogation between lateral layers of the network"""
        
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

In [45]:
def train_model(model, optimizer, imgs, labels, img_size, num_classes, num_epochs):
    """Bodied function to train data to network"""
    
    total_loss = 0.0
    loss = 0.0
    optimizer.zero_grad()
    # eval for train and test, use criterion and back propagation, specific towards
    # the type of loss that we want, contrastive
    for epoch in range(num_epochs):
        print("Epoch num: ", epoch)
        
        model = model.to(device)
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        for img, label in zip(imgs, labels):
            output = model(img)
            loss = criterion(output, label)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss
            
        print("loss for the epoch is:", total_loss)
        
    return model, total_loss

In [47]:
# Run time script
model = Model_()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# imgs = torch.from_numpy(imgs)
# labels = torch.from_numpy(labels)
print("Begin to train model ...")
m, loss = train_model(model, optimizer, imgs, labels, img_size, num_classes, num_epochs)
print("Done training model")
# TODO set up some graphs and visualization

Begin to train model ...
Epoch num:  0


RuntimeError: Expected 4-dimensional input for 4-dimensional weight 6 1 3 3, but got 1-dimensional input of size [784] instead